<h1>STXBP1 genotype-phenotype correlation  analysis</h1>
<p>todo</p>

In [1]:
import genophenocorr
print(f"Using genophenocorr version {genophenocorr.__version__}")

Using genophenocorr version 0.1.1dev


In [2]:
import hpotk

store = hpotk.configure_ontology_store()
hpo = store.load_minimal_hpo(release='v2023-10-09')
print(f'Loaded HPO v{hpo.version}')

Loaded HPO v2023-10-09


In [6]:
from genophenocorr.preprocessing import configure_caching_cohort_creator
cohort_creator = configure_caching_cohort_creator(hpo, timeout=20)

In [7]:
from genophenocorr.preprocessing import load_phenopacket_folder

In [11]:
phenopacket_input_folder = "phenopackets"
cohort = load_phenopacket_folder(pp_directory=phenopacket_input_folder, cohort_creator=cohort_creator)

Patients Created:  27%|██▋       | 126/462 [00:05<00:20, 16.39it/s]Expected a result but got an Error for variant: 9_127660109_127660109_G_T
{"error":"Could not connect to database homo_sapiens_core_111_38 as user ensro using [DBI:mysql:database=homo_sapiens_core_111_38;host=fb1-mysql-ens-rest-web.ebi.ac.uk;port=4571] as a locator:DBI connect('database=homo_sapiens_core_111_38;host=fb1-mysql-ens-rest-web.ebi.ac.uk;port=4571','ensro',...) failed: Can't connect to MySQL server on 'fb1-mysql-ens-rest-web.ebi.ac.uk' (111) at /nfs/public/ro/ensweb/live/rest/www_111/ensembl/modules/Bio/EnsEMBL/DBSQL/DBConnection.pm line 260."}
Patients Created:  37%|███▋      | 170/462 [00:09<00:17, 16.37it/s]Expected a result but got an Error for variant: 9_127653797_127653797_G_A
{"error":"Could not connect to database homo_sapiens_core_111_38 as user ensro using [DBI:mysql:database=homo_sapiens_core_111_38;host=fb1-mysql-ens-rest-web.ebi.ac.uk;port=4571] as a locator:DBI connect('database=homo_sapiens_cor

<h2>CohortViewer</h2>
<p>The CohortViewer class offers various methods to create tables and graphics to view the data in the cohort.
It is still very much in progress</p>

In [12]:
from genophenocorr.view import CohortViewer
from IPython.display import display, HTML

In [13]:
viewer = CohortViewer(hpo=hpo)

In [14]:
display(HTML(viewer.cohort_summary_table(cohort)))

Item,Description
Total Individuals,462
Excluded Individuals,7: STX_19557857_Patient_1[STX_19557857_Patient_1];STX_30182498_Patient_091[STX_30182498_Patient_091];STX_28944233_272650[STX_28944233_272650];STX_HSJD_Patient_14[STX_HSJD_Patient_14];STX_Syrbe_8[STX_Syrbe_8];STX_31780880_Patient1233[STX_31780880_Patient1233];STX_BCH_011[STX_BCH_011]
Total Unique HPO Terms,539
Total Unique Variants,455


In [15]:
display(HTML(viewer.variants_table(cohort, 'NM_003165.6')))

Variant,Effect,Count,Key
c.1217G>A,MISSENSE_VARIANT,19,9_127675910_127675910_G_A
c.1216C>T,MISSENSE_VARIANT,19,9_127675909_127675909_C_T
c.875G>A,MISSENSE_VARIANT,18,9_127668160_127668160_G_A
c.1651C>T,MISSENSE_VARIANT,18,9_127682509_127682509_C_T
c.416C>T,MISSENSE_VARIANT,12,9_127661192_127661192_C_T
c.568C>T,MISSENSE_VARIANT,11,9_127663343_127663343_C_T
c.1162C>T,STOP_GAINED,10,9_127675855_127675855_C_T
c.874C>T,MISSENSE_VARIANT,10,9_127668159_127668159_C_T
c.1439C>T,MISSENSE_VARIANT,10,9_127678510_127678510_C_T
c.364C>T,STOP_GAINED,9,9_127661140_127661140_C_T


In [17]:
display(HTML(viewer.hpo_term_counts_table(cohort)))

HPO Term,Count
Global developmental delay (HP:0001263),249
Absent speech (HP:0001344),151
Infantile spasms (HP:0012469),143
Seizure (HP:0001250),128
Hypsarrhythmia (HP:0002521),119
Infantile onset (HP:0003593),117
Neonatal onset (HP:0003623),115
"Intellectual disability, severe (HP:0010864)",105
Inability to walk (HP:0002540),93
Generalized tonic seizure (HP:0010818),91


In [18]:
cohort.list_data_by_tx('NM_003165.6')

{'NM_003165.6': Counter({'MISSENSE_VARIANT': 247,
          'STOP_GAINED': 71,
          'FRAMESHIFT_VARIANT': 68,
          'SPLICE_DONOR_VARIANT': 34,
          'SPLICE_ACCEPTOR_VARIANT': 21,
          'SPLICE_REGION_VARIANT': 18,
          'INFRAME_DELETION': 8,
          'SYNONYMOUS_VARIANT': 4,
          'THREE_PRIME_UTR_VARIANT': 2,
          'SPLICE_POLYPYRIMIDINE_TRACT_VARIANT': 1,
          'INTRON_VARIANT': 1})}

In [20]:
from genophenocorr.analysis import configure_cohort_analysis, CohortAnalysisConfiguration
from genophenocorr.model import VariantEffect

In [21]:
config = CohortAnalysisConfiguration()
config.pval_correction = 'fdr_bh'
analysis = configure_cohort_analysis(cohort, hpo, config=config)

In [25]:
from genophenocorr.analysis.predicate import PatientCategories

missense = analysis.compare_by_variant_effect(VariantEffect.MISSENSE_VARIANT, tx_id='NM_003165.6')
missense.summarize(hpo, category=PatientCategories.YES)

MISSENSE_VARIANT on NM_003165.6                       Yes            No  \
                                                    Count Percent Count   
Seizure precipitated by febrile infection [HP:0...  14/17     82%   3/7   
Febrile seizure (within the age range of 3 mont...  14/17     82%   3/7   
Infection-related seizure [HP:0032892]              14/17     82%   3/7   
Atonic seizure [HP:0010819]                         12/15     80%   3/7   
Focal impaired awareness autonomic seizure [HP:...    4/7     57%   0/4   
...                                                   ...     ...   ...   
Abnormality of the spleen [HP:0001743]                1/1    100%   0/0   
Hyporeflexia [HP:0001265]                             2/2    100%   0/0   
Focal myoclonic seizure [HP:0011166]                  5/8     62%   5/9   
Abnormal ankle physiology [HP:0034674]                2/2    100%   0/0   
Abnormal bone ossification [HP:0011849]               4/4    100%   0/0   

MISSENSE_VARIANT on NM_003165.6                                       \
                                                   Percent   p value   
Seizure precipitated by febrile infection [HP:0...     43%  0.133555   
Febrile seizure (within the age range of 3 mont...     43%  0.133555   
Infection-related seizure [HP:0032892]                 43%  0.133555   
Atonic seizure [HP:0010819]                            43%  0.144661   
Focal impaired awareness autonomic seizure [HP:...      0%  0.193939   
...                                                    ...       ...   
Abnormality of the spleen [HP:0001743]                  0%  1.000000   
Hyporeflexia [HP:0001265]                               0%  1.000000   
Focal myoclonic seizure [HP:0011166]                   56%  1.000000   
Abnormal ankle physiology [HP:0034674]                  0%  1.000000   
Abnormal bone ossification [HP:0011849]                 0%  1.000000   

MISSENSE_VARIANT on NM_003165.6                                       
                                                   Corrected p value  
Seizure precipitated by febrile infection [HP:0...               1.0  
Febrile seizure (within the age range of 3 mont...               1.0  
Infection-related seizure [HP:0032892]                           1.0  
Atonic seizure [HP:0010819]                                      1.0  
Focal impaired awareness autonomic seizure [HP:...               1.0  
...                                                              ...  
Abnormality of the spleen [HP:0001743]                           1.0  
Hyporeflexia [HP:0001265]                                        1.0  
Focal myoclonic seizure [HP:0011166]                             1.0  
Abnormal ankle physiology [HP:0034674]                           1.0  
Abnormal bone ossification [HP:0011849]                          1.0  

[983 rows x 6 columns]

Done for now.